Cél: Naponta bontott időjárási adatok letöltése több európai fővárosra 2020 és 2024 között.
A letöltött adatokat egyesítjük, hogy egy közös DataFrame-ben legyenek.
Ez szolgál majd bemenetként a mélytanuló modellhez, amely Budapest jövőbeli hőmérsékletét próbálja majd becsülni.

In [ ]:
#pip install scikit-learn

In [ ]:
#pip install meteostat

In [ ]:
#pip install geopy

In [ ]:
#pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [ ]:
#pip install matplotlib

In [ ]:
# Modellparaméterek
HIDDEN_SIZE = 32          # LSTM rejtett réteg mérete
NUM_LAYERS = 1            # LSTM rétegek száma
LEARNING_RATE = 0.001    # Tanulási ráta
BATCH_SIZE = 32           # Batch méret
N_STEPS = 7               # Rolling window méret (időlépések)

# Célváltozó
TARGET_COLUMN = "Budapest_tmax"

EARLY_STOPPING_PATIENCE = 10

In [ ]:
from geopy.geocoders import Nominatim
import time

# Például ezekre a városokra szeretnénk koordinátákat lekérni
cities = [
    "Budapest", "Vienna", "Bratislava", "Prague", "Warsaw",
    "Berlin", "Paris", "Rome", "Madrid", "Lisbon", "Athens", "Zagreb"
]

# Geolokátor inicializálása
geolocator = Nominatim(user_agent="weather_project")

# Üres szótár az eredményeknek
city_coordinates = {}

# Lekérdezések – fontos: aludjunk egy kicsit két hívás között, hogy ne blokkoljon az API
for city in cities:
    try:
        location = geolocator.geocode(city + ", Europe")  # opcionálisan szűkíthető
        if location:
            lat, lon = location.latitude, location.longitude
            city_coordinates[city] = (lat, lon)
            print(f"{city}: ({lat:.4f}, {lon:.4f})")
        else:
            print(f"{city}: Nem található.")
    except Exception as e:
        print(f"Hiba {city} lekérdezésekor: {e}")
    
    time.sleep(1)  # Nominatim kérések közötti várakozás (minimum 1 mp javasolt)

# Eredmények ellenőrzése
print(city_coordinates)


In [ ]:
from meteostat import Point, Daily
import pandas as pd
from datetime import datetime

# Időtartomány megadása: 2020. január 1. – 2024. december 31.
start = datetime(2000, 1, 1)
end = datetime(2024, 12, 31)

# Üres szótár az egyes városok adatainak tárolásához
all_data = {}

# Adatok lekérése minden városra külön
for name, (lat, lon) in city_coordinates.items():
    try:
        # Meteostat pozíció objektum létrehozása
        point = Point(lat, lon)
        
        # Napi időjárási adatok lekérése
        data = Daily(point, start, end).fetch()
        
        # Csak a szükséges oszlopokat tartjuk meg
        data = data[["tavg", "tmin", "tmax", "prcp", "wspd"]]
        
        # Oszlopok átnevezése a városnév hozzáadásával
        data.columns = [f"{name}_tavg", f"{name}_tmin", f"{name}_tmax", f"{name}_prcp", f"{name}_wspd"]
        
        # Adatok eltárolása a szótárban
        all_data[name] = data
        print(f"{name} adatai sikeresen letöltve.")
        
    except Exception as e:
        print(f"{name} adatainak lekérése sikertelen: {e}")

# Az összes város adatainak egyesítése közös időindex mentén
# Csak azok a napok maradnak meg, amelyek minden városnál rendelkezésre állnak
df = pd.concat(all_data.values(), axis=1, join='inner')
df.reset_index(inplace=True)

# Az eredmény exportálása CSV formátumban (opcionális)
df.to_csv("european_capitals_weather.csv", index=False)

# Az első néhány sor megjelenítése ellenőrzés céljából
df.head()


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split


In [ ]:
print("CUDA elérhető:", torch.cuda.is_available())
print("CUDA verzió:", torch.version.cuda)
print("PyTorch build CUDA-támogatással:", torch.backends.cudnn.enabled)
print("Alapértelmezett eszköz:", torch.device("cuda" if torch.cuda.is_available() else "cpu"))

In [ ]:
class WeatherDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).unsqueeze(1)  # shape: (n, 1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler

n_steps = N_STEPS  # Hány napos visszatekintés legyen

# Csak a feature-ök, a 'time' oszlop nélkül
raw_features = df.drop(columns=["time"])

# Skálázás DataFrame-ként, hogy később könnyen eldobjuk az oszlopokat
scaler = StandardScaler()
scaled_features = pd.DataFrame(scaler.fit_transform(raw_features), columns=raw_features.columns)

# NaN-t tartalmazó oszlopok eldobása
cleaned_features = scaled_features.dropna(axis=1)

# Numpy tömb formátumba visszaalakítás
features = cleaned_features.values

# Ellenőrzés
print("Maradt oszlop:", features.shape[1])

# A cél: Budapest tmax
target_column = "Budapest_tmax"
target_index = df.columns.get_loc(target_column) - 1  # -1, mert 'time' ki lett dobva
target_series = features[:, target_index]

# Rolling window input generálás
X = []
y = []

for i in range(n_steps, len(features)):
    X.append(features[i - n_steps:i])
    y.append(target_series[i])

X = np.array(X)
y = np.array(y)

# Célváltozók ellenőrzése - belefutottunk NaN értékekbe tanulítás során ezért megvizsgáljuk az adatokat
print("Célváltozó (y) statisztika:")
print("Y NaN:", np.isnan(y).any())
print("Y Inf:", np.isinf(y).any())
print("Y max:", np.max(y))
print("Y min:", np.min(y))
print(f"  Szórás: {np.std(y):.4f}")
print(f"  Átlag: {np.mean(y):.4f}")

print("Célváltozó (x) statisztika:")
print("X NaN:", np.isnan(X).any())
print("X Inf:", np.isinf(X).any())
print("X max:", np.max(X))
print("X min:", np.min(X))
print(f"  Szórás: {np.std(X):.4f}")
print(f"  Átlag: {np.mean(X):.4f}")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

train_dataset = WeatherDataset(X_train, y_train)
test_dataset = WeatherDataset(X_test, y_test)

# DataLoader-ek
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]  # csak az utolsó időlépés kimenetét használjuk
        out = self.fc(out)
        return out


In [ ]:
class EarlyStopping:
    def __init__(self, patience=EARLY_STOPPING_PATIENCE, verbose=True, delta=1e-4):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_loss = float('inf')
        self.early_stop = False
        self.delta = delta

    def __call__(self, val_loss):
        if val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.verbose:
                print(f"EarlyStopping: {self.counter}/{self.patience} epoch óta nincs javulás.", end=' ')
            if self.counter >= self.patience:
                self.early_stop = True


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Modell inicializálás
model = LSTMModel(input_size=X.shape[2], hidden_size=HIDDEN_SIZE, num_layers=NUM_LAYERS).to(device)


criterion = nn.MSELoss()
# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [ ]:
early_stopping = EarlyStopping(patience=10)
epoch = 0
train_loss_history = []

import time
start_time = time.time()

while epoch < 1000:  # maximum korlát, ha nincs early stop
    model.train()
    train_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)

    train_loss /= len(train_loader.dataset)
    print(f"\nEpoch {epoch+1}, Loss: {train_loss:.4f}", end=' ')
    train_loss_history.append(train_loss)


    early_stopping(train_loss)
    if early_stopping.early_stop:
        print("Korai leállítás aktiválva.")
        break

    epoch += 1

end_time = time.time()
elapsed_time = end_time - start_time  # másodpercben


In [ ]:
fname_suffix = f"H{HIDDEN_SIZE}_L{NUM_LAYERS}_LR{LEARNING_RATE}_BS{BATCH_SIZE}_S{N_STEPS}_PAT{EARLY_STOPPING_PATIENCE}"

csv_path = f"train_loss_{fname_suffix}.csv"
png_path = f"train_loss_{fname_suffix}.png"

In [ ]:
pd.DataFrame({
    "epoch": list(range(1, len(train_loss_history) + 1)),
    "train_loss": train_loss_history
}).to_csv(csv_path, index=False)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 4))
plt.plot(train_loss_history, label="Train loss")
plt.xlabel("Epoch")
plt.ylabel("Loss (MSE)")
plt.title("Tanulási görbe")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig(png_path)
plt.show()

In [ ]:
model.eval()
predictions = []
actuals = []

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs).cpu().numpy()
        predictions.extend(outputs.flatten())
        actuals.extend(targets.numpy().flatten())

from sklearn.metrics import mean_squared_error, mean_absolute_error
mse = mean_squared_error(actuals, predictions)
mae = mean_absolute_error(actuals, predictions)

print(f"Teszt MSE: {mse:.3f}")
print(f"Teszt MAE: {mae:.3f}")




In [ ]:
import os

log_file = "training_log.csv"
log_columns = [
    "training", "hidden_size", "learning_rate","layers", "epochs", "batch_size",
    "train_loss", "test_mse", "test_mae", "elapsed_time"
]

if not os.path.exists(log_file):
    pd.DataFrame(columns=log_columns).to_csv(log_file, index=False)

# Naplózás
log_entry = {
    "training": csv_path,
    "hidden_size": HIDDEN_SIZE,
    "learning_rate": LEARNING_RATE,
    "layers" : NUM_LAYERS,
    "epochs" : epoch + 1,
    "batch_size": BATCH_SIZE,
    "train_loss": train_loss,
    "test_mse": mse,
    "test_mae": mae,
    "elapsed_time": round(elapsed_time, 2)  # másodperc, 2 tizedesre kerekítve
}

# Napló hozzáfűzése
log_df = pd.read_csv(log_file)
log_df = pd.concat([log_df, pd.DataFrame([log_entry])], ignore_index=True)
log_df.to_csv(log_file, index=False)